Nota: Todas as chamadas de output_file() foram substituídas por output_notebook() para que os plots sejam apresentados em linha.

# O Básico de Bokeh

## Seu Primeiro Plot

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models.tools import LassoSelectTool, CrosshairTool, HoverTool
output_notebook()

x = [1, 3, 5, 7]
y = [2, 4, 6, 8]

p = figure()

p.circle(x, y, size=10, color='red', legend='círculo')
p.line(x, y, color='blue', legend='linha')
p.triangle(y, x, color='gold', size=10, legend='triângulo')

p.legend.click_policy='hide'

show(p)

# Bokeh e Pandas: Explorando o Dataset WWII THOR

## Carregando Dados no Pandas

In [ ]:
import pandas as pd

df = pd.read_csv('thor_wwii.csv')
print(df)

In [ ]:
df.columns.tolist()

## O Bokeh ColumnDataSource

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
output_notebook()

df = pd.read_csv('thor_wwii.csv')

amostra = df.sample(50)
fonte = ColumnDataSource(amostra)

p = figure()
p.circle(x='TOTAL_TONS', y='AC_ATTACKING',
         source=fonte,
         size=10, color='green')

p.title.text = 'Aeronave de Ataque e Munições Lançadas'
p.xaxis.axis_label = 'Toneladas de Munições Lançadas'
p.yaxis.axis_label = 'Número de Aeronaves de Ataque'

hover = HoverTool()
hover.tooltips=[
    ('Data do Ataque', '@MSNDATE'),
    ('Aeronave Atacante', '@AC_ATTACKING'),
    ('Toneladas de Munições', '@TOTAL_TONS'),
    ('Tipo de Aeronave', '@AIRCRAFT_NAME')
]

p.add_tools(hover)

show(p)

# Dados Categóricos e Gráficos de Barra: Munições Lançadas por País

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

from bokeh.palettes import Spectral5
from bokeh.transform import factor_cmap
output_notebook()

df = pd.read_csv('thor_wwii.csv')

agrupado = df.groupby('COUNTRY_FLYING_MISSION')['TOTAL_TONS', 'TONS_HE', 'TONS_IC', 'TONS_FRAG'].sum()

agrupado = agrupado / 1000

fonte = ColumnDataSource(agrupado)
paises = fonte.data['COUNTRY_FLYING_MISSION'].tolist()
p = figure(x_range=paises)

mapa_de_cores = factor_cmap(field_name='COUNTRY_FLYING_MISSION',
                    palette=Spectral5, factors=paises)

p.vbar(x='COUNTRY_FLYING_MISSION', top='TOTAL_TONS', source=fonte, width=0.70, color=mapa_de_cores)

p.title.text ='Munições Lançadas por País Aliado'
p.xaxis.axis_label = 'País'
p.yaxis.axis_label = 'Quilotons de Munição'

hover = HoverTool()
hover.tooltips = [
    ("Totais", "@TONS_HE Explosivo / @TONS_IC Incendiário / @TONS_FRAG Fragmentação")]

hover.mode = 'vline'

p.add_tools(hover)

show(p)

# Gráficos de Barras Empilhadas e Dados de Subamostragem: Tipos de Munições Lançadas por País

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3
output_notebook()

df = pd.read_csv('thor_wwii.csv')

filtro = df['COUNTRY_FLYING_MISSION'].isin(('USA','GREAT BRITAIN'))
df = df[filtro]

agrupado = df.groupby('COUNTRY_FLYING_MISSION')['TONS_IC', 'TONS_FRAG', 'TONS_HE'].sum()

#convertemos para quilotons novamente
agrupado = agrupado / 1000

fonte = ColumnDataSource(agrupado)
paises = fonte.data['COUNTRY_FLYING_MISSION'].tolist()
p = figure(x_range=paises)

p.vbar_stack(stackers=['TONS_HE', 'TONS_FRAG', 'TONS_IC'],
             x='COUNTRY_FLYING_MISSION', source=fonte,
             legend = ['Explosivo', 'Fragmentação', 'Incendiário'],
             width=0.5, color=Spectral3)

p.title.text ='Tipos de Munições Lançadas por País Aliado'
p.legend.location = 'top_left'

p.xaxis.axis_label = 'País'
p.xgrid.grid_line_color = None	#remove as linhas de grade de x

p.yaxis.axis_label = 'Quilotons de Munição'

show(p)

# Séries Temporais e Anotações: Operações de Bombardeio ao longo do Tempo

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3
output_notebook()

df = pd.read_csv('thor_wwii.csv')

#certifique-se de que MSNDATE tenha um formato de data
df['MSNDATE'] = pd.to_datetime(df['MSNDATE'], format='%m/%d/%Y')

agrupado = df.groupby('MSNDATE')['TOTAL_TONS', 'TONS_IC', 'TONS_FRAG'].sum()
agrupado = agrupado/1000

fonte = ColumnDataSource(agrupado)

p = figure(x_axis_type='datetime')

p.line(x='MSNDATE', y='TOTAL_TONS', line_width=2, source=fonte, legend='Todas as munições')
p.line(x='MSNDATE', y='TONS_FRAG', line_width=2, source=fonte, color=Spectral3[1], legend='Fragmentação')
p.line(x='MSNDATE', y='TONS_IC', line_width=2, source=fonte, color=Spectral3[2], legend='Incendiária')

p.yaxis.axis_label = 'Quilotons de Munições Lançadas'

show(p)

## Reamostragem de Dados de Série Temporal

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3
output_notebook()

df = pd.read_csv('thor_wwii.csv')

#certifique-se de que MSNDATE tenha um formato de data
df['MSNDATE'] = pd.to_datetime(df['MSNDATE'], format='%m/%d/%Y')

agrupado = df.groupby(pd.Grouper(key='MSNDATE', freq='M'))['TOTAL_TONS', 'TONS_IC', 'TONS_FRAG'].sum()
agrupado = agrupado/1000

fonte = ColumnDataSource(agrupado)

p = figure(x_axis_type='datetime')

p.line(x='MSNDATE', y='TOTAL_TONS', line_width=2, source=fonte, legend='Todas as munições')
p.line(x='MSNDATE', y='TONS_FRAG', line_width=2, source=fonte, color=Spectral3[1], legend='Fragmentação')
p.line(x='MSNDATE', y='TONS_IC', line_width=2, source=fonte, color=Spectral3[2], legend='Incendiária')

p.yaxis.axis_label = 'Quilotons de Munições Lançadas'

show(p)

# Anotando Tendências em Gráficos

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from datetime import datetime
from bokeh.palettes import Spectral3
output_notebook()

df = pd.read_csv('thor_wwii.csv')

#filtrar para o Teatro Europeu de Operações
filtro = df['THEATER']=='ETO'
df = df[filtro]

df['MSNDATE'] = pd.to_datetime(df['MSNDATE'], format='%m/%d/%Y')
agrupado = df.groupby(pd.Grouper(key='MSNDATE', freq='M'))['TOTAL_TONS', 'TONS_IC', 'TONS_FRAG'].sum()
agrupado = agrupado / 1000

fonte = ColumnDataSource(agrupado)

p = figure(x_axis_type="datetime")

p.line(x='MSNDATE', y='TOTAL_TONS', line_width=2, source=fonte, legend='Todas as Munições')
p.line(x='MSNDATE', y='TONS_FRAG', line_width=2, source=fonte, color=Spectral3[1], legend='Fragmentação')
p.line(x='MSNDATE', y='TONS_IC', line_width=2, source=fonte, color=Spectral3[2], legend='Incendiária')

p.title.text = 'Teatro Europeu de Operações'

p.yaxis.axis_label = 'Quilotons de Munições Lançadas'

show(p)

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.models import BoxAnnotation
from datetime import datetime
from bokeh.palettes import Spectral3
output_notebook()

df = pd.read_csv('thor_wwii.csv')

#filtrar para o Teatro Europeu de Operações
filtro = df['THEATER']=='ETO'
df = df[filtro]

df['MSNDATE'] = pd.to_datetime(df['MSNDATE'], format='%m/%d/%Y')
agrupado = df.groupby(pd.Grouper(key='MSNDATE', freq='M'))['TOTAL_TONS', 'TONS_IC', 'TONS_FRAG'].sum()
agrupado = agrupado / 1000

fonte = ColumnDataSource(agrupado)

p = figure(x_axis_type="datetime")

p.line(x='MSNDATE', y='TOTAL_TONS', line_width=2, source=fonte, legend='Todas as Munições')
p.line(x='MSNDATE', y='TONS_FRAG', line_width=2, source=fonte, color=Spectral3[1], legend='Fragmentação')
p.line(x='MSNDATE', y='TONS_IC', line_width=2, source=fonte, color=Spectral3[2], legend='Incendiária')

p.title.text = 'Teatro Europeu de Operações'

p.yaxis.axis_label = 'Quilotons de Munições Lançadas'

caixa_esquerda = pd.to_datetime('6-6-1944')
caixa_direita = pd.to_datetime('16-12-1944')

caixa = BoxAnnotation(left=caixa_esquerda, right=caixa_direita,
                    line_width=1, line_color='black', line_dash='dashed',
                    fill_alpha=0.2, fill_color='orange')

p.add_layout(caixa)

show(p)

# Dados Espaciais: Mapeando Localização de Alvos

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import layout
from bokeh.palettes import Spectral3
from bokeh.tile_providers import get_provider
from pyproj import Transformer
output_notebook()

# função auxiliar para converter latitude/longitude em easting/northing para mapeamento
# isso depende de funções da biblioteca pyproj


def LongLat_para_EN(long, lat):
    try:
        conversor = Transformer.from_crs('epsg:4326', 'epsg:3857')
        easting, northing = conversor.transform(long, lat)
        return easting, northing
    except:
        return None, None


df = pd.read_csv("thor_wwii.csv")


df['E'], df['N'] = zip(
    *df.apply(lambda x: LongLat_para_EN(x['TGT_LONGITUDE'], x['TGT_LATITUDE']), axis=1))

agrupado = df.groupby(['E', 'N'])[['TONS_IC', 'TONS_FRAG']].sum().reset_index()

filtro = agrupado['TONS_FRAG'] != 0
agrupado = agrupado[filtro]

fonte = ColumnDataSource(agrupado)

esquerda = -2150000
direita = 18000000
inferior = -5300000
superior = 11000000

p = figure(x_range=Range1d(esquerda, direita), y_range=Range1d(inferior, superior))

provedor = get_provider('CARTODBPOSITRON')
p.add_tile(provedor)

p.circle(x='E', y='N', source=fonte, line_color='grey', fill_color='yellow')

p.axis.visible = False

show(p)